In [132]:
import pandas as pd
import numpy as np 
import pickle 
import tensorflow as tf
from tensorflow.keras.models import load_model

In [133]:
## Load the trained mode, scaler pickle , OHE & labelencoder pickle
model = load_model('model.h5')

with open ('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)
    
with open ('onehot_encoder_geo.pkl','rb') as file:
    label_encoder_geo = pickle.load(file)
    
with open ('scaler.pkl','rb') as file:
    scaler = pickle.load(file)

In [134]:
print(scaler.feature_names_in_)

['CreditScore' 'Gender' 'Age' 'Tenure' 'Balance' 'NumOfProducts'
 'HasCrCard' 'IsActiveMember' 'EstimatedSalary' 'Geography_France'
 'Geography_Germany' 'Geography_Spain']


In [135]:
# Example input data 
input_data = {
    'CreditScore' : 600,
    'Geography' : 'France',
    'Gender' : 'Male',
    'Age' : 40,
    'Tenure' : 3,
    'Balance' : 60000,
    'NumOfProducts' : 2,
    'HasCrCard' : 1,
    'IsActiveMember' : 1,
    'EstimatedSalary' : 50000
}

In [136]:
# One-hot encode 'Geography'
geo_encoded = label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded,columns=label_encoder_geo.get_feature_names_out(['Geography']))

d:\ANN_End2End_Project\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [137]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [138]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [139]:
# Combine the one hot encoded coumns with input data

input_df = pd.concat([input_df.reset_index(drop=True),geo_encoded_df],axis=1)
input_df=input_df.drop('Geography',axis=1)

In [140]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [141]:
# Encode categorical variables 
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [142]:
## Scaling the data 
input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [143]:
# Predict churn 
prediction = model.predict(input_scaled)
print(prediction)

1/1 [==============================] - 0s 94ms/step
[[0.02069885]]


In [144]:
prediction_probab = prediction[0][0]
if prediction_probab>0.5:
    print("The customer is likely to churn.")
else:
    print("The customer is not likely to churn.")

The customer is not likely to churn.
